In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- Dense Layer inside a Dense Block ---
class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(DenseLayer, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, 4 * growth_rate, kernel_size=1, bias=False)

        self.bn2 = nn.BatchNorm2d(4 * growth_rate)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(4 * growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        out = self.conv1(self.relu1(self.bn1(x)))
        out = self.conv2(self.relu2(self.bn2(out)))
        out = torch.cat([x, out], 1)  # Concatenate input and output
        return out

In [ ]:
# --- Dense Block ---
class DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate):
        super(DenseBlock, self).__init__()
        layers = []
        for i in range(num_layers):
            layers.append(DenseLayer(in_channels + i * growth_rate, growth_rate))
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)

In [ ]:
# --- Transition Layer ---
class TransitionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionLayer, self).__init__()
        self.bn = nn.BatchNorm2d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.pool = nn.AvgPool2d(2, stride=2)

    def forward(self, x):
        out = self.conv(self.relu(self.bn(x)))
        out = self.pool(out)
        return out

In [ ]:

class DenseNet121(nn.Module):
    def __init__(self, growth_rate=32, num_classes=1000):
        super(DenseNet121, self).__init__()

        # Initial convolution
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(3, stride=2, padding=1)

        # Dense Blocks configuration for DenseNet-121
        block_layers = [6, 12, 24, 16]  # number of layers per dense block
        num_channels = 64

        # Dense Block 1
        self.block1 = DenseBlock(block_layers[0], num_channels, growth_rate)
        num_channels += block_layers[0] * growth_rate
        self.trans1 = TransitionLayer(num_channels, num_channels // 2)
        num_channels = num_channels // 2

        # Dense Block 2
        self.block2 = DenseBlock(block_layers[1], num_channels, growth_rate)
        num_channels += block_layers[1] * growth_rate
        self.trans2 = TransitionLayer(num_channels, num_channels // 2)
        num_channels = num_channels // 2

        # Dense Block 3
        self.block3 = DenseBlock(block_layers[2], num_channels, growth_rate)
        num_channels += block_layers[2] * growth_rate
        self.trans3 = TransitionLayer(num_channels, num_channels // 2)
        num_channels = num_channels // 2

        # Dense Block 4
        self.block4 = DenseBlock(block_layers[3], num_channels, growth_rate)
        num_channels += block_layers[3] * growth_rate

        # Classification layer
        self.bn2 = nn.BatchNorm2d(num_channels)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(num_channels, num_classes)

    def forward(self, x):
        out = self.pool1(self.relu(self.bn1(self.conv1(x))))

        out = self.trans1(self.block1(out))
        out = self.trans2(self.block2(out))
        out = self.trans3(self.block3(out))
        out = self.block4(out)

        out = self.avgpool(self.relu(self.bn2(out)))
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

"""
# --- Test DenseNet121 ---
if __name__ == "__main__":
    model = DenseNet121(num_classes=10)  # example: 10 classes
    x = torch.randn(1, 3, 224, 224)     # example input
    y = model(x)
    print(y.shape)  # should print torch.Size([1, 10])"""